<a href="https://colab.research.google.com/github/giacomogreggio/computer-vision-project/blob/master/VCS_Project_Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
import matplotlib.pyplot as plt
import tensorflow.keras.backend as back 
import cv2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input 

## VGG16

In [ ]:
initial_model = VGG16(weights="imagenet", include_top=False)
last = initial_model.output

cv1 = K.layers.Conv2D(4096, kernel_size=(7,7), activation='relu')(last)
cv2 = K.layers.Conv2D(4096, kernel_size=(1,1), activation='relu')(cv1)

cv3 = K.layers.Conv2D(21, kernel_size=(1,1), activation='relu')(cv2)
final = K.layers.Conv2DTranspose(21, kernel_size=(64,64), strides=(32,32), activation='softmax')(cv3)


model = K.Model(initial_model.input, final)

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   